In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
def prepare_data(csv_data):
    features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
    
    X = pd.get_dummies(csv_data[features]).fillna(-1)
    Y = csv_data.get("Survived")
    
    print(f"X:\n{X}\n\nY:\n{Y}")
    
    return np.array(X), np.array(Y)

In [5]:
print("Train data:")
train_data_X, train_data_Y = prepare_data(train_data)

Train data:
X:
     Pclass   Age  SibSp  Parch     Fare  Sex_female  Sex_male  Embarked_C  \
0         3  22.0      1      0   7.2500           0         1           0   
1         1  38.0      1      0  71.2833           1         0           1   
2         3  26.0      0      0   7.9250           1         0           0   
3         1  35.0      1      0  53.1000           1         0           0   
4         3  35.0      0      0   8.0500           0         1           0   
..      ...   ...    ...    ...      ...         ...       ...         ...   
886       2  27.0      0      0  13.0000           0         1           0   
887       1  19.0      0      0  30.0000           1         0           0   
888       3  -1.0      1      2  23.4500           1         0           0   
889       1  26.0      0      0  30.0000           0         1           1   
890       3  32.0      0      0   7.7500           0         1           0   

     Embarked_Q  Embarked_S  
0             0   

In [6]:
print("Test data:")
test_data_X, _ = prepare_data(test_data)

Test data:
X:
     Pclass   Age  SibSp  Parch      Fare  Sex_female  Sex_male  Embarked_C  \
0         3  34.5      0      0    7.8292           0         1           0   
1         3  47.0      1      0    7.0000           1         0           0   
2         2  62.0      0      0    9.6875           0         1           0   
3         3  27.0      0      0    8.6625           0         1           0   
4         3  22.0      1      1   12.2875           1         0           0   
..      ...   ...    ...    ...       ...         ...       ...         ...   
413       3  -1.0      0      0    8.0500           0         1           0   
414       1  39.0      0      0  108.9000           1         0           1   
415       3  38.5      0      0    7.2500           0         1           0   
416       3  -1.0      0      0    8.0500           0         1           0   
417       3  -1.0      1      1   22.3583           0         1           1   

     Embarked_Q  Embarked_S  
0      

In [7]:
from sklearn.model_selection import train_test_split

train_X, dev_X, train_Y, dev_Y = train_test_split(train_data_X, train_data_Y, test_size=0.3, random_state=42, stratify=train_data_Y)

train_Y = train_Y.reshape((-1, 1))
dev_Y = dev_Y.reshape((-1, 1))

print(train_X.shape, train_Y.shape)
print(dev_X.shape, dev_Y.shape)

(623, 10) (623, 1)
(268, 10) (268, 1)


In [8]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.1.0


In [9]:
from keras import Sequential
from keras.layers import Dense, Dropout

def get_model(input_size):
    model = Sequential([
        Dense(32, input_shape=(input_size,), activation="relu"),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dropout(0.2),
        Dense(1, activation="sigmoid")
    ])
    
    model.compile(optimizer="adam", metrics=["accuracy"], loss="binary_crossentropy")
    
    return model

Using TensorFlow backend.


In [10]:
model = get_model(train_X.shape[1])

In [11]:
model.fit(train_X, train_Y, epochs=1000, batch_size=10)

Epoch 1/1000
623/623 [==============================] - 0s 733us/step - loss: 1.9588 - accuracy: 0.5586
Epoch 2/1000
623/623 [==============================] - 0s 141us/step - loss: 1.2566 - accuracy: 0.5891
Epoch 3/1000
623/623 [==============================] - 0s 137us/step - loss: 0.9516 - accuracy: 0.6453
Epoch 4/1000
623/623 [==============================] - 0s 136us/step - loss: 0.8744 - accuracy: 0.6388
Epoch 5/1000
623/623 [==============================] - 0s 137us/step - loss: 0.7716 - accuracy: 0.6517
Epoch 6/1000
623/623 [==============================] - 0s 139us/step - loss: 0.7660 - accuracy: 0.6404
Epoch 7/1000
623/623 [==============================] - 0s 138us/step - loss: 0.7202 - accuracy: 0.6597
Epoch 8/1000
623/623 [==============================] - 0s 138us/step - loss: 0.7560 - accuracy: 0.6308
Epoch 9/1000
623/623 [==============================] - 0s 138us/step - loss: 0.6740 - accuracy: 0.6565
Epoch 10/1000
623/623 [==============================] - 0s 144u

In [12]:
_, accuracy = model.evaluate(dev_X, dev_Y, verbose=2)

print('\nDev accuracy:', accuracy)


Dev accuracy: 0.7947761416435242


In [13]:
predictions = model.predict(test_data_X)

predictions = np.round(predictions).astype(np.uint8).reshape((-1))

print(predictions)

[0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1
 1 0 0 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0
 0 0 1 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]


In [14]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)